#   _Key Derivation Functions_ - Funkcje wyprowadzania klucza
## LBK, Kollbi (Insert names later)
###   Podstawowe informacje
1.  Funkcje KDF to algorytmy kryptograficzne zwracające sekret lub wiele sekretów z tajnej wartości.    
        Np. Stworzenie klucza prywatnego (jeden sekret) z hasła użytkownika (tajna wartość).
1.  Metody te powstały ponieważ hasła użytkowników są przewidywalne i krótkie (niska entropia). Pojawia się potrzeba stworzenia mocnego klucza.  
Jednak nikt nie będzie pamiętał 64 znakowego hasła. Dlatego z pomocą przychodzi `key-stretching`.  
Proces w którym tworzony jest bezpieczniejszy i (teoretycznie) odporniejszy na ataki siłowe nowy klucz.  
`Key-streching` jest jedną z kluczowych technik zwiększania bezpieczeństwa klucza. Do key-streching'u można zaliczyć proste funkcje hashujące jak SHA, MD5 lub szyfry blokowe.
1. Funkcje wyprowadzania klucza powinny być `deterministyczne`. Oznacza to, że dla tych samych danych wejściowych otrzymamy zawsze ten sam skrót niezależnie ile razy funkcja zostanie wywołana.
1. Key Derivation Functions mogą być używane nie tylko do zapisywania haseł w postaci skrótów w bazie danych. Można je wykorzystywać między innymi do symetrycznego szyfrowania dysków lub kryptografi asymetrycznej.
```
                                               _             _  ______  _____                                   _       
     _ __   __ _ ___ _____      _____  _ __ __| |           | |/ /  _ \|  ___|           ___  ___  ___ _ __ ___| |_ ___  
    | '_ \ / _` / __/ __\ \ /\ / / _ \| '__/ _` |   =====\  | ' /| | | | |_     ====\   / __|/ _ \/ __| '__/ _ \ __/ __|
    | |_) | (_| \__ \__ \\ V  V / (_) | | | (_| |   =====/  | . \| |_| |  _|    ====/   \__ \  __/ (__| | |  __/ |_\__ \
    | .__/ \__,_|___/___/ \_/\_/ \___/|_|  \__,_|           |_|\_\____/|_|              |___/\___|\___|_|  \___|\__|___/
    |_|                                                     
```

### Funkcje KDF to nie tylko funkcje haszujące! Jednak mają wspólną część
1. Tak samo jak funkcje haszujące szyfrują dane deterministycznie. (To samo wejście - to samo wyjście)
1. Na wyjściu jest zawsze ciąg o takiej samej długości. Bez znaczenia jak duże były dane wejściowe.
1. Działa tylko w jedną stronę. Nie odzyska wejścia danych z wyjścia.


### Zrozumienie od podstaw

Najprostszą funkcję wyprowadzenia klucza można stworzyć wykorzystując dowolny algorytm haszujący. Dla przykładu użyty zostanie algorytm SHA256.  

function(`password`, `sha256`) --> `key`

In [5]:
from hashlib import sha256

hash = sha256(b"password")

print(hash.digest().hex())

5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8


Należy pamiętać, że powyższe rozwiązanie nie jest bezpieczne. Proste hasze są podatne na ataki słownikowe (np. tęczowe tablice (Rainbow table)). Jak zatem zwiększyć bezpieczeństwo?

Pierwszym krokiem jest dodanie `soli` (ang. `salt`).  
`Salt` jest wartością przechowywaną razem z uzyskanym kluczem. Używana, żeby utrudnić atak siłowy np. Rainbow Table. Sprawia, że nowy hasz jest bardziej skomplikowany i zmniejsza szansę pojawienia się go we wcześniej obliczonych tęczowych tablicach.

function_with_salt(`salt`, `password`, `sha256`) --> `key`  

In [6]:
password = b"password"
salt = b"salt"
hash = sha256(salt+password)

print(hash.digest().hex())

13601bda4ea78e55a07b98866d2be6be0744e3866f13c00c811cab608a28f322


Funkcję wyżej można opisać jako `HKDF` (`HMAC-based key derivation function`) - najprostsza funkcja wyprowadzania klucza.  
Ciągle jest mniej bezpieczna niż aktualne KDF. Zaleca się korzystanie z PBKDF2, Bcrypt, Scrypt (ten projekt) oraz Argon2.



### Przykład 1.0
Wykorzystanie `HMAC` (Hash-based message authentication code) z solą, hasłem użytkownika oraz funkcją SHA256 do pozyskania klucza.
1. Sprawdź co się stanie jak zmienisz sól. Może to być bardzo mała zmiana. Porównaj parę wyników dla tego samego hasła. 
1. Czy klucze z nową solą znacznie różnią się od głównego? Jeśli tak to dlaczego. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))
1. Porównaj dwa klucze z tym samym hasłem w następujący sposób:  
    1. Dla samej funkcji haszującej `sha256(b"password")`
    1. Następnie stwórz dwie różne sole.
    1. Porównaj klucze dołączając sól po haśle `sha256(b"password+salt")`

In [4]:
from hmac import HMAC
from hashlib import sha256

salt = b'NaCl'
password = b'CryptoIsFun'

key = HMAC(salt, password, digestmod=sha256)

print(key.digest().hex())  # Desired output: `bff0e9482b9412e77f6b401c421957921782ff2f385521498ce59357fdc6da72`

bff0e9482b9412e77f6b401c421957921782ff2f385521498ce59357fdc6da72


### Zasosowanie `soli`
Wielokrotne użycie soli dla tego samego hasła umożliwia pozyskanie wielu innych kluczy. Jest to często stosowane rozwiązanie.  
Dla przykładu: można posiadać w bazie dwóch użytkowników z takim samym hasłem. Zwykłe zahaszowanie hasła w obu przypadkach zwróci to samo.  
Jednak wygenerowanie dla każdego z nich unikalnej wartości (`salt`) i dołączenie do hasła, sprawia że otrzymane hasze są diametralnie różne. ([Lawinowość](https://en.wikipedia.org/wiki/Avalanche_effect))  
KDF pomimo znajomości klucza z `salt1` nie jest wstanie znaleźć klucza z `salt2`.

### Po podstawach

Wiedząc co rozróżnia funkcje haszujące od KDF oraz czym jest sól, zwrócona zostanie uwaga na to, co powinna zapewniać dobra funkcja wyprowadzania klucza.  

1. Memory Hardness - cecha funkcji zapewniająca, że wymaga dużej ilości pamięci do obliczeń lub wymaga znacznie większego nakładu czasowego, tak by maszyny wykonujące obliczenia równoległe nie były znacząco bardziej efektywne w obliczeniach od standardowych procesorów CPU.  
Algorytm określa się "memory-hard" gdy potrzeba wysokiej ilości pamięci do pojedynczego wykonania. 

1. Funkcja KDF, która ma być wysoce odporna na ataki siłowe lub słownikowe, powinna maksymalnie spowolnić możliwość obliczeń równoległych. Jednocześnie wykonując relatywnie szybkie obliczenie klucza bądź skrótu.

1. Żeby "opóźnić" generację kluczy można zwiększyć złożoność obliczeniową, złożoność pamięciową lub oba jednocześnie.

Więcej informacji: `Funkcje wyprowadzania klucza (KDF) wykorzystujące odwzorowanie logistyczne - Grzegorz Frejek`

### Wprowadzenie `scrypt`

`Scrypt` jest funkcją wyprowadzania klucza opartą na haśle. Implementacja opiera się na funkcjach, które zapewniają dodatkową ochronę (memory hardness).

Scrypt posiada więcej parametrów niż stworzony przez nas wcześniej przykład. 
Parametry scrypt są następujące:
1. `Passphrase`
1. `Salt`
1. `Block size`
1. `CPU/Memory cost` (>1; potęga 2; < 2^(128 * r / 8))
1. `Parallelization parametr` (>0; <= ((2^32-1) * 32) / (128 * r))
1. `Key length` (długość w oktetach; >0; <= (2^32 - 1) * 32)

Najpierw należy zaznajomić się z poszczególnymi funkcjami algorytmu. A potem działanie algorytmu można przedstawić w 3 krokach.

### Salsa20/8 - wariant z [Salsa20](https://en.wikipedia.org/wiki/Salsa20)

Salsa20/8 nie może być uznana za funkcję haszującą ponieważ nie jest odporna na kolizje.
`Salsa20/8` oraz `Salsa20/12` są wariantami z `Salsa20`. Nie zostały stworzone, żeby wyprzec oryginalną implementację. Wręcz przeciwnie. Mają za zadanie dopełniać i działać lepiej.  

Algorytm wykonuje operacje `XOR` na ciągach znaków w określonej liczbie rund. Salsa20 wykonuje 20 rund. Salsa20/8 i Salsa20/12 odpowiednio po 8 i 12 rund.

<!-- Można wstawić jakiś ładny opis. Troszkę bardziej rozbudowany. Co to robi czy coś. Chociaż kod mówi sam za siebie. -->
Zmodyfikowany algorytm w języku C znajduje się niżej.

```c
 #define R(a,b) (((a) << (b)) | ((a) >> (32 - (b))))
   void salsa20_word_specification(uint32 out[16],uint32 const in[16])
   {
     int i;
     uint32 x[16];
     for (i = 0;i < 16;++i) x[i] = in[i];
     for (i = 8;i > 0;i -= 2) {
       x[ 4] ^= R(x[ 0]+x[12], 7);  x[ 8] ^= R(x[ 4]+x[ 0], 9);
       x[12] ^= R(x[ 8]+x[ 4],13);  x[ 0] ^= R(x[12]+x[ 8],18);
       x[ 9] ^= R(x[ 5]+x[ 1], 7);  x[13] ^= R(x[ 9]+x[ 5], 9);
       x[ 1] ^= R(x[13]+x[ 9],13);  x[ 5] ^= R(x[ 1]+x[13],18);
       x[14] ^= R(x[10]+x[ 6], 7);  x[ 2] ^= R(x[14]+x[10], 9);
       x[ 6] ^= R(x[ 2]+x[14],13);  x[10] ^= R(x[ 6]+x[ 2],18);
       x[ 3] ^= R(x[15]+x[11], 7);  x[ 7] ^= R(x[ 3]+x[15], 9);
       x[11] ^= R(x[ 7]+x[ 3],13);  x[15] ^= R(x[11]+x[ 7],18);
       x[ 1] ^= R(x[ 0]+x[ 3], 7);  x[ 2] ^= R(x[ 1]+x[ 0], 9);
       x[ 3] ^= R(x[ 2]+x[ 1],13);  x[ 0] ^= R(x[ 3]+x[ 2],18);
       x[ 6] ^= R(x[ 5]+x[ 4], 7);  x[ 7] ^= R(x[ 6]+x[ 5], 9);
       x[ 4] ^= R(x[ 7]+x[ 6],13);  x[ 5] ^= R(x[ 4]+x[ 7],18);
       x[11] ^= R(x[10]+x[ 9], 7);  x[ 8] ^= R(x[11]+x[10], 9);
       x[ 9] ^= R(x[ 8]+x[11],13);  x[10] ^= R(x[ 9]+x[ 8],18);
       x[12] ^= R(x[15]+x[14], 7);  x[13] ^= R(x[12]+x[15], 9);
       x[14] ^= R(x[13]+x[12],13);  x[15] ^= R(x[14]+x[13],18);
     }
     for (i = 0;i < 16;++i) out[i] = x[i] + in[i];
   }
```

W ostatniej lini zmieszana tablica jest dodawana słowo po słowie, żeby oryginalna tablica uzyskała wielkość 64-bitowych bloków.  Jest to bardzo ważne ponieważ dodanie do oryginalnej tablicy zmieszancyh wartości sprawia, że niemożliwe jest pozyskanie danych wejściowych. Dokładnie ta sama technika jest szeroko wykorzystywana w funkcjach haszujących.

### Algorytm scryptBlockMix
Scrypt wykorzystuje `scryptBlockMix` jako funkcję haszującą.
Reprezentacja algorytmu:
<!-- Tłumaczenie? -->
```bash
Parameters:
  r       Block size parameter.

  Input:
      B[0] || B[1] || ... || B[2 * r - 1]
          Input octet string (of size 128 * r octets),
          treated as 2 * r 64-octet blocks,
          where each element in B is a 64-octet block.

  Output:
      B\'[0] || B\'[1] || ... || B\'[2 * r - 1]
        Output octet string.

  Steps:

    1. X = B[2 * r - 1]

    2. for i = 0 to 2 * r - 1 do
        T = X xor B[i]
        X = Salsa (T)
        Y[i] = X
      end for

    3. B\' = (Y[0], Y[2], ..., Y[2 * r - 2], Y[1],
              Y[3], ..., Y[2 * r - 1])
```

### Algorytm scryptROMix

Algorytm sROMix wykorzystuje scryptBlockMix jako funkcję haszującą `H` oraz funkcję `Integerify` (dosłowne tłumaczenie - ??? całkowicie???)
<!-- Again tłumaczenie ? -->
Reprezentacja algorytmu:  
```bash
Input:
        r       Block size parameter.
        B       Input octet vector of length 128 * r octets.
        N       CPU/Memory cost parameter, must be larger than 1,
                a power of 2, and less than 2^(128 * r / 8).

Output:
        B\'      Output octet vector of length 128 * r octets.

Steps:

    1. X = B

    2. for i = 0 to N - 1 do
        V[i] = X
        X = scryptBlockMix (X)
    end for

    3. for i = 0 to N - 1 do
        j = Integerify (X) mod N
                where Integerify (B[0] ... B[2 * r - 1]) is defined
                as the result of interpreting B[2 * r - 1] as a
                little-endian integer.
        T = X xor V[j]
        X = scryptBlockMix (T)
    end for

    4. B\' = X
```



### Scrypt
Wracając do początku sekcji - algorytm można przedstawić w 3 krokach.
Zapis PBKDF2-HMAC-SHA-256 oznacza, że został użyty algorytm PBKDF2 (Password-Based Key Derivation Function) z HMAC-SHA-256 jako PRF (Pseudorandom Function).

<!-- 
#TODO Przetłumaczyć to jakoś mądrze. (To co wyżej też) (╭☞￢ Ĺ̯￢ )╭☞. (Albo zostawić ang. bo jest bardziej cool.) Z tego korzystałem. 6 punkt i idziesz w górę ^ https://datatracker.ietf.org/doc/html/rfc7914#page-3 
-->
Algorytm:  

```
1.  Initialize an array B consisting of p blocks of 128 * r octets
        each:
            B[0] || B[1] || ... || B[p - 1] = PBKDF2-HMAC-SHA256 (P, S, 1, p * 128 * r)

2.  for i = 0 to p - 1 do
        B[i] = scryptROMix (r, B[i], N)
    end for

3.  DK = PBKDF2-HMAC-SHA256 (P, B[0] || B[1] || ... || B[p - 1], 1, dkLen)
```

# #TODO Omówienie algorytmu? W którym miejscu wykazuje on własność Memmory Hardness.

